# Puzzle 8: Seven Segment Search

This puzzle seems much more challenging than those that came before.

In [113]:
filename = "input8.txt"
with open(filename,"r") as infile:
    input = [line.strip().split("|") for line in infile]

print(input[0])

['eadbcf faceb faecgd gdefabc adc ad adbf gfacbe bceda dcegb ', ' gdfcae adc cedbfa dafb']


In [114]:
signalPatterns = [pattern[0].split() for pattern in input]
outputValues = [value[1].split() for value in input]
combinedData = list(zip(signalPatterns, outputValues))
print(signalPatterns[0], outputValues[0])
print(combinedData[0])

['eadbcf', 'faceb', 'faecgd', 'gdefabc', 'adc', 'ad', 'adbf', 'gfacbe', 'bceda', 'dcegb'] ['gdfcae', 'adc', 'cedbfa', 'dafb']
(['eadbcf', 'faceb', 'faecgd', 'gdefabc', 'adc', 'ad', 'adbf', 'gfacbe', 'bceda', 'dcegb'], ['gdfcae', 'adc', 'cedbfa', 'dafb'])


### Part 1
The seven segment displays of 1   4   7   8, are unique in that they each are the only values to use 2,4,3,7 segments to display their value on the seven segment display.

In [115]:
# count each of the occurences of these values in output values (method 1)
count = 0
for value in combinedData[0][1]:
    if len(value) in (2,3,4,7):
        print(value, len(value))
        count += 1
count

adc 3
dafb 4


2

In [116]:
# method 2, use a dictionary (or map as its called cpp)

# convert the list of output values to a flat list
oneDimensionalOutputvalues = [value for row in outputValues for value in row]
# list of the lengths of each output value using map func
lengthsOfOutputValues = list(map(lambda x: len(x), oneDimensionalOutputvalues))

# count the total occurences of each length (so occurences of unique seven segment digits)
from collections import Counter as counter
countsOfLengths = counter(lengthsOfOutputValues) # {}

total = 0
for k,v in countsOfLengths.items():
    if k in (2,3,4,7): # the unique digits
        total += v
        # print(k, v)
print(f"In the output values, the digits 1, 4, 7, or 8 appear a total of <{total}> times") # answer to part 1


In the output values, the digits 1, 4, 7, or 8 appear a total of <539> times


### Part 2
Determine the remaining digits

In [117]:
#   0:     
#  aaaa    
# b    c  
# b    c  
#  dddd   
# e    f  
# e    f  
#  gggg 

# load in the example from question
testSignalPatterns = 'acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab'.split()
testOutputValues = "cdfeb fcadb cdfeb cdbaf".split()
print(testSignalPatterns, testOutputValues, sep="\n\n")

['acedgfb', 'cdfbe', 'gcdfa', 'fbcad', 'dab', 'cefabd', 'cdfgeb', 'eafb', 'cagedb', 'ab']

['cdfeb', 'fcadb', 'cdfeb', 'cdbaf']


Puzzle input consists of essentially line after line of the same type of scrambled codes. Each one is scrambled similarly as described in the puzzle description, but each line is a unique 'cypher', if it can be called that.

Solved the example problem in the question, on paper. Next step is to design a set of functions to do this, so it can be performed quickly on every single line of the puzzle input.

In [118]:
lettersList = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
signalMapping = dict(list(zip(lettersList,[None]*10)))
print(signalMapping)

{'a': None, 'b': None, 'c': None, 'd': None, 'e': None, 'f': None, 'g': None}


In [119]:
# possibly remove decode approach and reuse signal mapping variable name for number translated

# finds the encoded numbers (1,4,8,7) by looking at their unique lenghts of (2,4,7,3) respectively
# returns dictionary of numbers (keys) and encoded strings (values)
def find_unique_lengths(signalPatterns):
    numbersTranslated = dict(list(zip(list(range(0,10)),[None]*10)))
    uniqueLengths = {2:1, 4:4, 7:8, 3:7} # {length:integer represented}

    for item in signalPatterns:
        if len(item) in uniqueLengths.keys():
            numbersTranslated[uniqueLengths[len(item)]] = item

    return numbersTranslated

numbersTranslated = find_unique_lengths(testSignalPatterns)
numbersTranslated

{0: None,
 1: 'ab',
 2: None,
 3: None,
 4: 'eafb',
 5: None,
 6: None,
 7: 'dab',
 8: 'acedgfb',
 9: None}

The variable numbers translated will be used to compare to the output values to find out what integer they represent, in the puzzle input.

In [125]:
# find the top segment signal code
top = [i for i in numbersTranslated[7] if i not in numbersTranslated[1]][0]
top

'd'

Integer 1 is important as it contains only UR and LR segments. We can look at the three different signals with six segments. The pattern that does not contain both signals in the signal for 1, must be 6. This is because the three integers with 6 segments are 0, 6, and 9. 0 and 9 both incorporate the same segments as 1. 

In [153]:
# Find all integers that use 6 segments, by filtering signalPatterns for len(6)
def find_six(signalPatterns: list, numbersTranslated: dict ):
    one = numbersTranslated[1]
    signalsOfLengthSix = [s for s in signalPatterns if len(s) == 6]
    six = [s for s in signalsOfLengthSix if one not in s][0]
    UR = [s for s in list(one) if s not in list(six)][0]
    BR = list(one)
    return six, UR, BR

six, UR, BR = find_six(testSignalPatterns, numbersTranslated)
# add pattern for six to numberTranslated
numbersTranslated[6] = six
print(six, UR, BR)



cdfgeb a ['a', 'b']


Find the middle segment, which allows us to distinguish between 0 and 9 integer signal patterns. We do this by looking at the remaining length 6 integer signal patterns and finding the signal characters (a ~ g, excluding the known characters), of which there are three. Comparing to the pattern for 8, the only pattern of length 7, we will see one pattern contains a unique character - this it the BL. 

In addition if we compare the (now) remaining two patterns, with only three characters remaining in each, one of the characters will also be present in all other signal patterns of length five (integers 2,3,5), this is the middle segment.

As a consequence of this we will know the signal pattern of 9 since in the initial set of patterns of length 6, it is the only one to lack the BL segment. Thus 0 will be the other pattern. 

In [ ]:
def find_zero_and_nine()

The functions below are called on the output values after the | demlimiter

In [122]:
# # for testing
# encodedlist = ['d', 'e', 'a', 'f', 'g', 'b', 'c']
# signalMapping = dict(list(zip(lettersList,encodedlist))
# encodedNumberTest = 'cdfeb' # => abdfg => 5

In [123]:
# may not need to use this

# takes an encoded number string and returns decodedNumber as list in alphabetical order
def decode_letters_string(encodedNumber: str, signalMapping: dict):
    return ''.join(sorted([signalMapping[letter] for letter in list(encodedNumber)]))

# decodedNumber = decode_letters_string(encodedNumberTest, signalMapping)

In [124]:
# takes string input decoded number which is a signal to the display
# and returns the integer value it represents
def translate_2_seven_segment_num(decodedNumber):
    NormalisedSignalsList = ['abcefg', 'cf', 'acdeg', 'acdfg', 'bcdf', 'abdfg', 'abdefg', 'acf', 'abcdefg', 'abcdfg']
    lettersTranslateDict = dict(list(zip(NormalisedSignalsList,list(range(0,10)))))

    if decodedNumber in lettersTranslateDict.keys():
        return lettersTranslateDict[decodedNumber]
    else:
        ValueError

# translate_2_seven_segment_num(decodedNumber)